# 8: Constrained Travel Time Ratio 

Model 8 takes an alternative approach in valuing the different times by imposing a constraint on the relative importance of OVT and IVT. This is achieved by replacing the travel time variables in the modal utility equations with a weighted travel time (WTT) variable defined as in-vehicle time plus the appropriate travel time importance ratio (TIR) times out-of-vehicle time (IVT + TIR×OVT). The parameter for out-of-vehicle time is equal to the parameter for in-vehicle time multiplied by the selected travel time ratio (TTR).

In Model 8, the TTR is 2.5. (pp. 114)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#2,3") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#2,3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +P("nonmotorized_time") * X("(altid>4) * tottime")
    + P("motorized_ivtt") * (X("(altid <= 4) * ivtt") + 2.5 * X("(altid <= 4) * ovtt"))
    + PX("totcost")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 8, TTR = 2.5"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*ivtt.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

The value of motorized_ovtt is the TTR * motorized_ivtt. In this instance, it is 2.5 * (-0.0254) = -0.0635.

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_BIKE": -1.8019897052221918,
    "ASC_SR2": -2.3298554039662283,
    "ASC_SR3P": -3.756413794251253,
    "ASC_TRAN": -0.582221916159616,
    "ASC_WALK": 0.4380372173849091,
    "hhinc#2,3": -0.0015674849574736722,
    "hhinc#4": -0.005515844848425141,
    "hhinc#5": -0.012357167709979168,
    "hhinc#6": -0.009464085538793363,
    "motorized_ivtt": -0.02536879896953912,
    "nonmotorized_time": -0.06627353731450113,
    "totcost": -0.00486979028829524,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )